In [1]:
import glob
import cv2 as cv
import numpy as np
import pandas as pd
list_area = []
list_area2 = []
blob_dia = []
blob_dia2 = []
est_tail_len = []
splatter = []
images = [cv.imread(file) for file in glob.glob(r"C:\Users\SHUBHAM THORAT\Documents\Manu_project\Images\**\*.bmp")]
# images,
for i in range(len(images)):
    img = images[i]
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret ,bw = cv.threshold(img, 80, 255, cv.THRESH_BINARY)
    im=cv.countNonZero(bw)
    list_area.append(im*64)
    #print(im)
    bw2 = ~bw
    ret ,bw1 = cv.threshold(img, 40, 255, cv.THRESH_BINARY)
    im1=cv.countNonZero(bw1)
    list_area2.append(im1*64)
    bw_low = ~bw1
    #print(f"im1" , im1)

    # create the params and deactivate the 3 filters
    params = cv.SimpleBlobDetector_Params()
    params2 = cv.SimpleBlobDetector_Params()
    # params.filterByArea = False
    # params.filterByInertia = False
    # params.filterByConvexity = False

    # params.maxArea = 100000
    # params.minInertiaRatio = 0.1
    # params.minConvexity = 0.1


    # Filter by Area.
    params.filterByArea = False
    params2.filterByArea = True
    params2.minArea = 10

    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.6
    params2.filterByCircularity = True
    params2.minCircularity = 0
    # Filter by Convexity
    params.filterByConvexity = False
    params2.filterByConvexity = False
    params.minConvexity = 0.87

    # Filter by Inertia
    params.filterByInertia = False
    params2.filterByInertia = False
    params.minInertiaRatio = 0.01

    # Set up the detector with default parameters.
    detector = cv.SimpleBlobDetector_create(params)
    detector2 = cv.SimpleBlobDetector_create(params2)
    
    # Detect blobs.
    keypoints = detector.detect(bw2)
    keypoints2 = detector2.detect(bw_low)
    
    # Draw detected blobs as red circles.
    # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
    im_with_keypoints = cv.drawKeypoints(bw, keypoints, np.array([]), (0,0,255), cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    im_with_keypoints2 = cv.drawKeypoints(bw1, keypoints2, np.array([]), (0,0,255), cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    temp = [0]
    for kp in keypoints: 
        temp.append(kp.size)
    blob_dia.append(max(temp))

    if len(keypoints2) > 1: splatter.append(1)
    else : splatter.append(0)
    
    temp = [0]
    for kp2 in keypoints2:
        temp.append(kp2.size)    
    blob_dia2.append(max(temp))
    tail_est = blob_dia2[-1] - blob_dia[-1]
    est_tail_len.append(tail_est*8)
    # a = img - bw

    # splatter = bool
    # if im1-img != 0:
    #     splatter = True
    # else:
    #     splatter = False

    # print(splatter)

    # contours,hierarchy = cv.findContours(bw, 1, 2)
    # cnt = contours[0]

    # (x,y),radius = cv.minEnclosingCircle(cnt)  
    # center = (int(x),int(y))
    # radius = int(radius)
    # cv.circle(bw,center,radius,(0,255,0),5)
    
    #cv.imshow('Image', im_with_keypoints)
    #cv.imshow('Image2', im_with_keypoints2)
    #cv.imshow('Image1', key)
    #cv.imshow('ImageOG', img)
    cv.waitKey(0)
    cv.destroyAllWindows()
# for saving files in different directory https://pythonguides.com/python-save-an-image-to-file/




In [2]:
print(len(list_area))
print(len(blob_dia))
print(len(blob_dia2))
print(len(splatter)) 
print(len(est_tail_len))

163887
163887
163887
163887
163887


In [9]:
frame_img = pd.DataFrame(list(zip(list_area,est_tail_len, splatter)), columns = ['Area','Tail length', 'splatter'])

In [10]:
frame_img.to_csv('Frame_img.csv', index = False)